In [1]:
# Tutorial from here: http://blog.yhat.com/posts/harry-potter-classification.html?imm_mid=0efe5b&cmp=em-data-na-na-newsltr_20170405

# Import Modules
import pandas as pd
import requests

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-1-0bb09c24d755>, line 7)

In [4]:
# Get Ravenclaw articles
category = 'Ravenclaws'
url = 'http://harrypotter.wikia.com/api/v1/Articles/List?expand=1&limit=1000&category=' + category

requested_url = requests.get(url)
json_results = requested_url.json()
info = json_results['items']
ravenclaw_df = pd.DataFrame(info)

In [6]:
json_results

{'basepath': 'http://harrypotter.wikia.com',
 'items': [{'abstract': '{{Ravenclaw individual...',
   'comments': 0,
   'id': 5080,
   'ns': 10,
   'original_dimensions': None,
   'revision': {'id': 964956,
    'timestamp': '1460047333',
    'user': 'SuperSajuuk',
    'user_id': 1251315},
   'thumbnail': None,
   'title': 'Ravenclaw individual infobox',
   'url': '/wiki/Template:Ravenclaw_individual_infobox'},
  {'abstract': 'Roland Abberley was a Ravenclaw student at Hogwarts School of Witchcraft and Wizardry in the...',
   'comments': 0,
   'id': 33946,
   'ns': 0,
   'original_dimensions': None,
   'revision': {'id': 1024340,
    'timestamp': '1479282062',
    'user': 'Ironyak1-bot',
    'user_id': 29343889},
   'thumbnail': None,
   'title': 'Roland Abberley',
   'type': 'article',
   'url': '/wiki/Roland_Abberley'},
  {'abstract': 'Stewart Ackerley (born c. 1982-1983) was a wizard who began attending Hogwarts School of...',
   'comments': 0,
   'id': 7011,
   'ns': 0,
   'original_

In [7]:
print('Number of articles: {}'.format(len(info)))
print('')
ravenclaw_df.head()

Number of articles: 159



,abstract,comments,id,ns,original_dimensions,revision,thumbnail,title,type,url
0,{{Ravenclaw individual...,0,5080,10,None,"{'id': 964956, 'user': 'SuperSajuuk', 'user_id...",None,Ravenclaw individual infobox,NaN,/wiki/Template:Ravenclaw_individual_infobox
1,Roland Abberley was a Ravenclaw student at Hog...,0,33946,0,None,"{'id': 1024340, 'user': 'Ironyak1-bot', 'user_...",None,Roland Abberley,article,/wiki/Roland_Abberley
2,Stewart Ackerley (born c. 1982-1983) was a wiz...,0,7011,0,None,"{'id': 1024309, 'user': 'Ironyak1-bot', 'user_...",None,Stewart Ackerley,article,/wiki/Stewart_Ackerley
3,Jatin Agarkar was a Ravenclaw student at Hogwa...,0,99467,0,None,"{'id': 1039350, 'user': 'Rodolphus', 'user_id'...",None,Jatin Agarkar,article,/wiki/Jatin_Agarkar
4,Alannis was a female Ravenclaw student at Hogw...,0,27126,0,"{'width': 322, 'height': 546}","{'id': 1024320, 'user': 'Ironyak1-bot', 'user_...",https://vignette3.wikia.nocookie.net/harrypott...,Alannis,article,/wiki/Alannis


In [8]:
# Set variables
# Set variables
houses = ['Gryffindor', 'Hufflepuff', 'Ravenclaw', 'Slytherin']
mydf = pd.DataFrame()

# Gets article ids, article url, and house
for house in houses:
    url = "http://harrypotter.wikia.com/api/v1/Articles/List?expand=1&limit=1000&category=" + house + 's'
    requested_url = requests.get(url)
    json_results = requested_url.json()
    info = json_results['items']

    house_df = pd.DataFrame(info)
    house_df = house_df[house_df['type'] == 'article']
    house_df.reset_index(drop=True, inplace=True)
    house_df.drop(['abstract', 'comments', 'ns', 'original_dimensions', 'revision', 'thumbnail', 'type'], axis=1, inplace=True)
    house_df['house'] = pd.Series([house]*len(house_df))
    mydf = pd.concat([mydf, house_df])

mydf.reset_index(drop=True, inplace=True)

# Print results
print('Number of student articles: {}'.format(len(mydf)))
print('')
print(mydf.head())
print('')
print(mydf.tail())

Number of student articles: 750

      id             title                     url       house
0  33349     Astrix Alixan     /wiki/Astrix_Alixan  Gryffindor
1  33353   Filemina Alchin   /wiki/Filemina_Alchin  Gryffindor
2   7018  Euan Abercrombie  /wiki/Euan_Abercrombie  Gryffindor
3  99282      Sakura Akagi      /wiki/Sakura_Akagi  Gryffindor
4  99036       Zakir Akram       /wiki/Zakir_Akram  Gryffindor

         id             title                     url      house
745  100562  Phylis Whitehead  /wiki/Phylis_Whitehead  Slytherin
746    3153            Wilkes            /wiki/Wilkes  Slytherin
747   35971      Ella Wilkins      /wiki/Ella_Wilkins  Slytherin
748   44393    Rufus Winickus    /wiki/Rufus_Winickus  Slytherin
749     719     Blaise Zabini     /wiki/Blaise_Zabini  Slytherin


In [9]:
# Loops through articles and pulls the "Personality and traits" section from each student
# If that section does not exist for a student, we just report a blank string
# This takes a few minutes to run
text_dict = {}
for iden in mydf['id']:
    url = 'http://harrypotter.wikia.com/api/v1/Articles/AsSimpleJson?id=' + str(iden)
    requested_url = requests.get(url)
    json_results = requested_url.json()
    sections = json_results['sections']
    contents = [sections[i]['content'] for i, x in enumerate(sections) if sections[i]['title'] == 'Personality and traits']

    if contents:
        paragraphs = contents[0]
        texts = [paragraphs[i]['text'] for i, x in enumerate(paragraphs)]
        all_text = ' '.join(texts)
    else:
        all_text = ''
    text_dict[iden] = all_text

# Places data into a DataFrame and computes the length of the "Personality and traits" section
text_df = pd.DataFrame.from_dict(text_dict, orient='index')
text_df.reset_index(inplace=True)
text_df.columns = ['id', 'text']
text_df['text_len'] = text_df['text'].map(lambda x: len(x))

# Merges our text data back with the info about the students
mydf_all = pd.merge(mydf, text_df, on='id')
mydf_all.sort_values('text_len', ascending=False, inplace=True)

In [10]:
# Creates a new DataFrame with just the students who have a "Personality and traits" section
mydf_relevant = mydf_all[mydf_all['text_len'] > 0]

print('Number of useable articles: {}'.format(len(mydf_relevant)))
print('')
mydf_relevant.head()

Number of useable articles: 95



,id,title,url,house,text,text_len
691,343,Tom Riddle,/wiki/Tom_Riddle,Slytherin,"Voldemort was considered by many to be ""the mo...",26924
169,13,Harry Potter,/wiki/Harry_Potter,Gryffindor,"Harry was an extremely brave, loyal, and selfl...",13207
728,49,Dolores Umbridge,/wiki/Dolores_Umbridge,Slytherin,"Dolores Umbridge was an evil woman, nothing sh...",9683
705,259,Horace Slughorn,/wiki/Horace_Slughorn,Slytherin,Horace Slughorn was described as having a bumb...,8277
54,4178,Albus Dumbledore,/wiki/Albus_Dumbledore,Gryffindor,Considered to be the most powerful wizard of h...,7789


In [11]:
trait_dict = {}
trait_dict['Gryffindor'] = ['bravery', 'nerve', 'chivalry', 'daring', 'courage']
trait_dict['Slytherin'] = ['resourcefulness', 'cunning', 'ambition', 'determination', 'self-preservation', 'fraternity',
                           'cleverness']
trait_dict['Ravenclaw'] = ['intelligence', 'wit', 'wisdom', 'creativity', 'originality', 'individuality', 'acceptance']
trait_dict['Hufflepuff'] = ['dedication', 'diligence', 'fairness', 'patience', 'kindness', 'tolerance', 'persistence',
                            'loyalty']

In [12]:
from nltk.corpus import wordnet as wn

# Synsets of differents words
foo1 = wn.synsets('bravery')
print("Synonym sets associated with the word 'bravery': {}".format(foo1))

foo2 = wn.synsets('fairness')
print('')
print("Synonym sets associated with the word 'fairness': {}".format(foo2))

foo3 = wn.synsets('wit')
print('')
print("Synonym sets associated with the word 'wit': {}".format(foo3))

foo4 = wn.synsets('cunning')
print('')
print("Synonym sets associated with the word 'cunning': {}".format(foo4))

foo4 = wn.synsets('cunning', pos=wn.NOUN)
print('')
print("Synonym sets associated with the *noun* 'cunning': {}".format(foo4))
print('')

# Prints out the synonyms ("lemmas") associated with each synset
foo_list = [foo1, foo2, foo3, foo4]
for foo in foo_list:
    for synset in foo:
        print((synset.name(), synset.lemma_names()))

LookupError: 
**********************************************************************
  Resource 'corpora/wordnet' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - 'C:\\Users\\ABaker/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\ABaker\\AppData\\Local\\Continuum\\Anaconda3\\nltk_data'
    - 'C:\\Users\\ABaker\\AppData\\Local\\Continuum\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\ABaker\\AppData\\Roaming\\nltk_data'
**********************************************************************